In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import glob
import os
import csv
import numpy as np
import matplotlib
import re
from nlp import NLPTransforms
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import seaborn as sns
import plotly.plotly as py
sns.set(style="darkgrid", font_scale=1.5)
%matplotlib inline

In [2]:
from sklearn.datasets import fetch_20newsgroups

In [3]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))

In [4]:
lookup = {}
for k in range(20):
    lookup[k] = dataset.target_names[k]
lookup

{0: 'alt.atheism',
 1: 'comp.graphics',
 2: 'comp.os.ms-windows.misc',
 3: 'comp.sys.ibm.pc.hardware',
 4: 'comp.sys.mac.hardware',
 5: 'comp.windows.x',
 6: 'misc.forsale',
 7: 'rec.autos',
 8: 'rec.motorcycles',
 9: 'rec.sport.baseball',
 10: 'rec.sport.hockey',
 11: 'sci.crypt',
 12: 'sci.electronics',
 13: 'sci.med',
 14: 'sci.space',
 15: 'soc.religion.christian',
 16: 'talk.politics.guns',
 17: 'talk.politics.mideast',
 18: 'talk.politics.misc',
 19: 'talk.religion.misc'}

In [5]:
reverse_lookup = {v: k for k, v in lookup.items()}

In [6]:
documents = dataset.data

In [7]:
targets = dataset.target

In [8]:
# Cleaning
Clean = NLPTransforms()
documents = list(map(lambda x: Clean.clean_str(Clean.lowercase_strip_punc(x)), documents))

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
def dirtify(df, frac=0.05, categories=20):
    df_copy = df.copy()
    sample = df_copy.sample(frac=frac)
    for idx, row in sample.iterrows():
        df_copy['label'][idx] = np.random.choice([x for x in range(categories) if x != sample.loc[idx]['label']])
    df['dirty_label'] = df_copy['label']
    return df

In [11]:
import gensim
class CorpusFriendly(object):
    def __init__(self, df):
        self.data_frame = df
            
    def __iter__(self):
        for idx, row in self.data_frame.iterrows():
            yield gensim.models.doc2vec.TaggedDocument(row['doc'].split(), [lookup[row['dirty_label']]])

2017-06-22 13:31:32,147 : INFO : 'pattern' package not found; tag filters are not available for English


In [12]:
doc_train, doc_test, target_train, target_test = train_test_split(documents, targets, test_size = 0.33)
train = pd.DataFrame({'doc':doc_train, 'label': target_train})

def train_with_noise(train, frac):
    df = dirtify(train, frac=frac)
    corpus = CorpusFriendly(df)
    model = gensim.models.doc2vec.Doc2Vec(size=500, min_count=5, window=8, iter=15, workers=4)
    model.build_vocab(corpus)
    model.train(corpus, total_examples=model.corpus_count, epochs=model.iter)
    return model, df

In [13]:
def accuracy(model, doc_test, target_test):
    results = []
    scores = []
    for i in range(len(doc_test)):
        inferred_vector = model.infer_vector(doc_test[i].split())
        sims = model.docvecs.most_similar([inferred_vector], topn=1)
        results.append(reverse_lookup[sims[0][0]])
        scores.append(sims[0][1])
        #print(doc_test[n])
        #print(sims)
        #print(lookup[target_test[n]])
    
    df_result = pd.DataFrame({'doc': doc_test, 'label': target_test, 'results': results, 'scores': scores})
    acc = (df_result['label'] == df_result['results']).sum() / df_result.shape[0]
    
    return acc, df_result

In [14]:
import timy

def get_acc(frac, output):
    model, df = train_with_noise(train, frac=frac)
    acc, df_result = accuracy(model, doc_test, target_test)
    output.put(acc)

In [15]:
def get_test(frac, output):
    result = frac + 2
    output.put(result)

In [39]:
import multiprocessing as mp


iter_num =3
frac_list = [0.05, 0.1, 0.15, 0.2, 0.25]


@timy.timer(ident='multiprocess_avg', loops=1)
def multiprocess_avg(frac, final, iter_num=3):
    
    output = mp.Queue()
    processes = [mp.Process(target=get_acc, args=(f, output)) for f in [frac]*iter_num]

    for p in processes:
        p.start()
    
    for p in processes:
        p.join()
    
    results = [output.get() for p in processes]
    #return sum(results)/ float(len(results))
    final.put(results)

In [22]:
final = mp.Queue()
multiprocess_avg(frac=0.05, final)
print(results)

/Users/Eddie/Documents/DSSG/WDI/nosiy_label_learning/venv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/Eddie/Documents/DSSG/WDI/nosiy_label_learning/venv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/Eddie/Documents/DSSG/WDI/nosiy_label_learning/venv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

2017-06-22 13:36:08

2017-06-22 13:36:23,619 : INFO : PROGRESS: at 17.73% examples, 193443 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:36:23,727 : INFO : PROGRESS: at 17.73% examples, 191960 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:36:24,501 : INFO : PROGRESS: at 19.14% examples, 195637 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:36:24,646 : INFO : PROGRESS: at 18.96% examples, 192489 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:36:24,766 : INFO : PROGRESS: at 19.06% examples, 191936 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:36:25,542 : INFO : PROGRESS: at 20.28% examples, 193748 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:36:25,685 : INFO : PROGRESS: at 20.11% examples, 191206 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:36:25,819 : INFO : PROGRESS: at 20.23% examples, 190579 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:36:26,589 : INFO : PROGRESS: at 21.74% examples, 193386 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:36:26,695 : INFO : PROGRESS: at 21.51% examples, 191166 wor

2017-06-22 13:36:51,145 : INFO : PROGRESS: at 53.14% examples, 190073 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:36:51,304 : INFO : PROGRESS: at 53.22% examples, 189902 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:36:51,315 : INFO : PROGRESS: at 52.76% examples, 187920 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:36:52,148 : INFO : PROGRESS: at 54.52% examples, 189811 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:36:52,328 : INFO : PROGRESS: at 53.93% examples, 187531 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:36:52,345 : INFO : PROGRESS: at 54.68% examples, 189606 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:36:53,176 : INFO : PROGRESS: at 55.77% examples, 189830 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:36:53,339 : INFO : PROGRESS: at 55.37% examples, 187823 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:36:53,356 : INFO : PROGRESS: at 55.89% examples, 189580 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:36:54,230 : INFO : PROGRESS: at 57.40% examples, 190026 wor

2017-06-22 13:37:17,947 : INFO : PROGRESS: at 87.79% examples, 189505 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:37:18,817 : INFO : PROGRESS: at 90.05% examples, 191680 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:37:18,857 : INFO : PROGRESS: at 89.53% examples, 190679 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:37:18,972 : INFO : PROGRESS: at 89.10% examples, 189639 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:37:19,834 : INFO : PROGRESS: at 91.40% examples, 191721 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:37:19,871 : INFO : PROGRESS: at 90.97% examples, 190767 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:37:19,993 : INFO : PROGRESS: at 90.62% examples, 189657 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:37:20,871 : INFO : PROGRESS: at 92.69% examples, 191866 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:37:20,911 : INFO : PROGRESS: at 92.23% examples, 190871 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:37:20,995 : INFO : PROGRESS: at 91.86% examples, 189990 wor

My calculation executed (multiprocess_avg) for 1 time in 105.823261 seconds seconds
My calculation best time was 105.823261 seconds seconds
0.579182288877


In [37]:
frac_list = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]
@timy.timer(ident='Multiprocessing', loops=1)
def multiprocess_evaluate(frac_list=frac_list):
    final = mp.Queue() 
    processes = [mp.Process(target=multiprocess_avg, args=(frac, final)) for frac in frac_list]
    
    for p in processes:
        p.start()
    
    for p in processes:
        p.join()
    
    results = [final.get() for p in processes]
    
    return results
    

In [38]:
r = multiprocess_evaluate()
print(r)

/Users/Eddie/Documents/DSSG/WDI/nosiy_label_learning/venv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/Eddie/Documents/DSSG/WDI/nosiy_label_learning/venv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/Eddie/Documents/DSSG/WDI/nosiy_label_learning/venv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/Eddie/Docume

2017-06-22 14:40:45,782 : INFO : collecting all words and their counts
2017-06-22 14:40:45,816 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-06-22 14:40:52,210 : INFO : collected 77139 word types and 20 unique tags from a corpus of 7580 examples and 1331525 words
2017-06-22 14:40:52,244 : INFO : Loading a fresh vocabulary
2017-06-22 14:40:52,404 : INFO : collected 77139 word types and 20 unique tags from a corpus of 7580 examples and 1331525 words
2017-06-22 14:40:52,431 : INFO : Loading a fresh vocabulary
2017-06-22 14:40:52,482 : INFO : collected 77139 word types and 20 unique tags from a corpus of 7580 examples and 1331525 words
2017-06-22 14:40:52,506 : INFO : Loading a fresh vocabulary
2017-06-22 14:40:52,609 : INFO : min_count=5 retains 14878 unique words (19% of original 77139, drops 62261)
2017-06-22 14:40:52,653 : INFO : min_count=5 leaves 1242706 word corpus (93% of original 1331525, drops 88819)
2017-06-22 14:40:52,832 : INFO : min_coun

2017-06-22 14:41:13,480 : INFO : PROGRESS: at 10.68% examples, 84599 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:41:14,082 : INFO : PROGRESS: at 11.07% examples, 83913 words/s, in_qsize 8, out_qsize 0
2017-06-22 14:41:14,152 : INFO : PROGRESS: at 10.97% examples, 83349 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:41:14,535 : INFO : PROGRESS: at 11.10% examples, 83862 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:41:15,086 : INFO : PROGRESS: at 11.55% examples, 83703 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:41:15,253 : INFO : PROGRESS: at 11.49% examples, 83101 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:41:15,553 : INFO : PROGRESS: at 11.55% examples, 83617 words/s, in_qsize 8, out_qsize 0
2017-06-22 14:41:16,154 : INFO : PROGRESS: at 12.15% examples, 83599 words/s, in_qsize 8, out_qsize 0
2017-06-22 14:41:16,297 : INFO : PROGRESS: at 11.97% examples, 83084 words/s, in_qsize 8, out_qsize 0
2017-06-22 14:41:16,562 : INFO : PROGRESS: at 12.10% examples, 83404 words/s, in_q

2017-06-22 14:41:42,227 : INFO : PROGRESS: at 26.60% examples, 82813 words/s, in_qsize 8, out_qsize 0
2017-06-22 14:41:42,416 : INFO : PROGRESS: at 27.03% examples, 83209 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:41:42,918 : INFO : PROGRESS: at 27.10% examples, 83269 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:41:43,242 : INFO : PROGRESS: at 27.21% examples, 82851 words/s, in_qsize 8, out_qsize 0
2017-06-22 14:41:43,507 : INFO : PROGRESS: at 27.78% examples, 83407 words/s, in_qsize 8, out_qsize 0
2017-06-22 14:41:43,940 : INFO : PROGRESS: at 27.72% examples, 83272 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:41:44,276 : INFO : PROGRESS: at 27.96% examples, 83019 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:41:44,617 : INFO : PROGRESS: at 28.47% examples, 83530 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:41:45,059 : INFO : PROGRESS: at 28.40% examples, 83397 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:41:45,285 : INFO : PROGRESS: at 28.53% examples, 83123 words/s, in_q

2017-06-22 14:42:10,657 : INFO : PROGRESS: at 43.60% examples, 84246 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:42:10,875 : INFO : PROGRESS: at 44.11% examples, 84649 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:42:11,283 : INFO : PROGRESS: at 44.11% examples, 84684 words/s, in_qsize 8, out_qsize 0
2017-06-22 14:42:11,732 : INFO : PROGRESS: at 44.25% examples, 84230 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:42:11,978 : INFO : PROGRESS: at 44.69% examples, 84635 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:42:12,340 : INFO : PROGRESS: at 44.74% examples, 84730 words/s, in_qsize 8, out_qsize 1
2017-06-22 14:42:12,777 : INFO : PROGRESS: at 44.82% examples, 84308 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:42:13,088 : INFO : PROGRESS: at 45.26% examples, 84694 words/s, in_qsize 8, out_qsize 0
2017-06-22 14:42:13,368 : INFO : PROGRESS: at 45.25% examples, 84773 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:42:13,818 : INFO : PROGRESS: at 45.39% examples, 84391 words/s, in_q

...

2017-06-22 14:58:32,573 : INFO : PROGRESS: at 52.62% examples, 138456 words/s, in_qsize 8, out_qsize 0
2017-06-22 14:58:32,901 : INFO : PROGRESS: at 52.78% examples, 138876 words/s, in_qsize 8, out_qsize 1
2017-06-22 14:58:33,228 : INFO : PROGRESS: at 53.09% examples, 138522 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:58:33,582 : INFO : PROGRESS: at 53.69% examples, 138830 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:58:33,935 : INFO : PROGRESS: at 53.88% examples, 139210 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:58:34,229 : INFO : PROGRESS: at 54.12% examples, 138609 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:58:34,645 : INFO : PROGRESS: at 54.85% examples, 139106 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:58:34,948 : INFO : PROGRESS: at 55.04% examples, 139492 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:58:35,276 : INFO : PROGRESS: at 55.32% examples, 138872 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:58:35,716 : INFO : PROGRESS: at 55.90% examples, 139201 wor

My calculation executed (multiprocess_avg) for 1 time in 1194.634361 seconds seconds
My calculation best time was 1194.634361 seconds seconds


2017-06-22 14:58:42,932 : INFO : PROGRESS: at 63.52% examples, 140616 words/s, in_qsize 8, out_qsize 0
2017-06-22 14:58:43,310 : INFO : PROGRESS: at 63.96% examples, 141334 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:58:43,585 : INFO : PROGRESS: at 64.19% examples, 140913 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:58:43,991 : INFO : PROGRESS: at 64.90% examples, 141680 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:58:44,326 : INFO : PROGRESS: at 65.21% examples, 142305 words/s, in_qsize 8, out_qsize 0
2017-06-22 14:58:44,591 : INFO : PROGRESS: at 65.48% examples, 141912 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:58:45,010 : INFO : PROGRESS: at 66.26% examples, 142480 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:58:45,331 : INFO : PROGRESS: at 66.47% examples, 143080 words/s, in_qsize 8, out_qsize 0
2017-06-22 14:58:45,610 : INFO : PROGRESS: at 66.78% examples, 142691 words/s, in_qsize 8, out_qsize 0
2017-06-22 14:58:46,075 : INFO : PROGRESS: at 67.72% examples, 143294 wor

2017-06-22 14:59:10,287 : INFO : PROGRESS: at 97.66% examples, 153086 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:59:10,713 : INFO : PROGRESS: at 98.66% examples, 154000 words/s, in_qsize 8, out_qsize 0
2017-06-22 14:59:11,049 : INFO : PROGRESS: at 98.88% examples, 154263 words/s, in_qsize 8, out_qsize 0
2017-06-22 14:59:11,289 : INFO : PROGRESS: at 98.64% examples, 153221 words/s, in_qsize 7, out_qsize 0
2017-06-22 14:59:11,688 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-06-22 14:59:11,727 : INFO : PROGRESS: at 99.93% examples, 154371 words/s, in_qsize 2, out_qsize 1
2017-06-22 14:59:11,732 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-06-22 14:59:11,756 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-06-22 14:59:11,788 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-06-22 14:59:11,801 : INFO : training on 19972875 raw words (14623630 effective words) took 94.7s, 154378 effective w

My calculation executed (multiprocess_avg) for 1 time in 1236.147039 seconds seconds
My calculation best time was 1236.147039 seconds seconds
Multiprocessing executed (multiprocess_evaluate) for 1 time in 1236.248503 seconds seconds
Multiprocessing best time was 1236.248503 seconds seconds
[0.57623638635957863, 0.57712908409212638, 0.55650776647027322, 0.55008034279592921, 0.53901089091233712, 0.51892519193001252, 0.50035707909301907, 0.48250312444206389]


In [48]:
import threading 
import time
exitFlag = 0
class myThread(threading.Thread):
    def __init__(self, threadID, name, frac):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.frac = frac
        self.acc = None
    def run(self):
        print("Starting " + self.name)
        self.acc = get_acc(self.frac)
        print("Exiting " + self.name)
        
iter_num =3
frac_list = [0.05]*iter_num

@timy.timer(ident='My calculation', loops=1)
def multithread_acc(frac_list):
    threads = []
    for n, frac in enumerate(frac_list):
        threads.append(myThread(n, "Thread-{}".format(frac), frac))

    for thread in threads:
        thread.start()

    for t in threads:
        t.join()
    print ("Exiting Main Thread: ")
    return threads


In [49]:
t = multithread_acc(frac_list)
avg_acc = sum([t[x].acc for x in range(len(t))]) / float(len(t))
print("Average: ", avg_acc)

Starting Thread-0.05Starting Thread-0.05

Starting Thread-0.05


/Users/Eddie/Documents/DSSG/WDI/nosiy_label_learning/venv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

2017-06-22 13:00:14,865 : INFO : collecting all words and their counts
2017-06-22 13:00:15,803 : INFO : collecting all words and their counts
2017-06-22 13:00:16,052 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-06-22 13:00:16,124 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-06-22 13:00:16,705 : INFO : collecting all words and their counts
2017-06-22 13:00:16,914 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-06-22 13:00:20,069 : INFO : collected 75903 word types and 20 unique tags from a corpus of 7580 examples and 1392037 words
2017-06-22 13:00:20,

2017-06-22 13:00:38,298 : INFO : PROGRESS: at 13.38% examples, 139959 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:00:38,524 : INFO : PROGRESS: at 17.06% examples, 156473 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:00:39,317 : INFO : PROGRESS: at 14.36% examples, 141086 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:00:39,327 : INFO : PROGRESS: at 15.71% examples, 144159 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:00:39,524 : INFO : PROGRESS: at 17.94% examples, 156579 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:00:40,321 : INFO : PROGRESS: at 15.31% examples, 140169 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:00:40,360 : INFO : PROGRESS: at 16.58% examples, 143608 words/s, in_qsize 8, out_qsize 1
2017-06-22 13:00:40,597 : INFO : PROGRESS: at 19.08% examples, 156382 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:00:41,330 : INFO : PROGRESS: at 16.04% examples, 138738 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:00:41,440 : INFO : PROGRESS: at 17.59% examples, 144106 wor

2017-06-22 13:01:06,176 : INFO : PROGRESS: at 37.30% examples, 134155 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:01:06,692 : INFO : PROGRESS: at 38.60% examples, 134729 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:01:06,878 : INFO : PROGRESS: at 40.76% examples, 138371 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:01:07,241 : INFO : PROGRESS: at 38.21% examples, 134589 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:01:07,695 : INFO : PROGRESS: at 39.48% examples, 134465 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:01:07,995 : INFO : PROGRESS: at 41.66% examples, 138116 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:01:08,262 : INFO : PROGRESS: at 39.14% examples, 134556 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:01:08,766 : INFO : PROGRESS: at 40.35% examples, 134120 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:01:09,000 : INFO : PROGRESS: at 42.67% examples, 138230 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:01:09,308 : INFO : PROGRESS: at 40.04% examples, 134200 wor

2017-06-22 13:01:34,071 : INFO : PROGRESS: at 64.61% examples, 137111 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:01:34,309 : INFO : PROGRESS: at 62.44% examples, 135059 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:01:34,990 : INFO : PROGRESS: at 62.67% examples, 132535 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:01:35,095 : INFO : PROGRESS: at 65.31% examples, 136605 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:01:35,319 : INFO : PROGRESS: at 63.42% examples, 135347 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:01:36,012 : INFO : PROGRESS: at 63.63% examples, 132692 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:01:36,135 : INFO : PROGRESS: at 66.16% examples, 136346 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:01:36,356 : INFO : PROGRESS: at 64.38% examples, 135665 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:01:37,024 : INFO : PROGRESS: at 64.40% examples, 132798 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:01:37,157 : INFO : PROGRESS: at 67.07% examples, 136203 wor

2017-06-22 13:02:01,823 : INFO : PROGRESS: at 86.26% examples, 133217 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:02:02,090 : INFO : PROGRESS: at 88.93% examples, 135576 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:02:02,570 : INFO : PROGRESS: at 87.58% examples, 135414 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:02:02,836 : INFO : PROGRESS: at 87.26% examples, 133318 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:02:03,130 : INFO : PROGRESS: at 89.90% examples, 135687 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:02:03,694 : INFO : PROGRESS: at 88.33% examples, 135072 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:02:03,883 : INFO : PROGRESS: at 88.06% examples, 133109 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:02:04,132 : INFO : PROGRESS: at 90.73% examples, 135640 words/s, in_qsize 7, out_qsize 0
2017-06-22 13:02:04,759 : INFO : PROGRESS: at 89.58% examples, 135439 words/s, in_qsize 8, out_qsize 0
2017-06-22 13:02:04,914 : INFO : PROGRESS: at 89.09% examples, 133304 wor

Exiting Thread-0.05
Exiting Thread-0.05
Exiting Thread-0.05
Exiting Main Thread: 
My calculation executed (multithread_acc) for 1 time in 170.572181 seconds seconds
My calculation best time was 170.572181 seconds seconds
Average:  0.580253526156


In [ ]:
[t[x].acc for x in range(len(t))]

In [50]:
@timy.timer(ident='My calculation', loops=1)
def normal_acc_avg(frac, iter_num=3):
    acc_avg = 0
    for i in range(iter_num):
        print('\t Iter {}...'.format(i+1))
        model, df = train_with_noise(train, frac=frac)
        acc, df_result = accuracy(model, doc_test, target_test)
        #acc = get_test(frac)
        acc_avg += acc
    return acc_avg / float(iter_num)

In [51]:
normal_acc_avg(frac=0.05, iter_num=3)

	 Iter 1...


/Users/Eddie/Documents/DSSG/WDI/nosiy_label_learning/venv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

2017-06-22 13:02:57,976 : INFO : collecting all words and their counts
2017-06-22 13:02:57,978 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-06-22 13:02:58,995 : INFO : collected 75903 word types and 20 unique tags from a corpus of 7580 examples and 1392037 words
2017-06-22 13:02:58,995 : INFO : Loading a fresh vocabulary
2017-06-22 13:02:59,057 : INFO : min_count=5 retains 15096 unique words (19% of original 75903, drops 60807)
2017-06-22 13:02:59,057 : INFO : min_count=5 leaves 1304889 word corpus (93% of original 1392037, drops 87148)
2017-06-22 13:02:59,099 : INFO : deleting the raw counts dictionary of 75903 items
2017-06

	 Iter 2...


2017-06-22 13:03:55,071 : INFO : collecting all words and their counts
2017-06-22 13:03:55,072 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-06-22 13:03:56,066 : INFO : collected 75903 word types and 20 unique tags from a corpus of 7580 examples and 1392037 words
2017-06-22 13:03:56,066 : INFO : Loading a fresh vocabulary
2017-06-22 13:03:56,126 : INFO : min_count=5 retains 15096 unique words (19% of original 75903, drops 60807)
2017-06-22 13:03:56,126 : INFO : min_count=5 leaves 1304889 word corpus (93% of original 1392037, drops 87148)
2017-06-22 13:03:56,171 : INFO : deleting the raw counts dictionary of 75903 items
2017-06-22 13:03:56,176 : INFO : sample=0.001 downsamples 45 most-common words
2017-06-22 13:03:56,177 : INFO : downsampling leaves estimated 1011849 word corpus (77.5% of prior 1304889)
2017-06-22 13:03:56,178 : INFO : estimated required memory for 15096 words and 500 dimensions: 67976000 bytes
2017-06-22 13:03:56,227 : INFO : rese

	 Iter 3...


2017-06-22 13:04:54,919 : INFO : collecting all words and their counts
2017-06-22 13:04:54,921 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-06-22 13:04:55,891 : INFO : collected 75903 word types and 20 unique tags from a corpus of 7580 examples and 1392037 words
2017-06-22 13:04:55,892 : INFO : Loading a fresh vocabulary
2017-06-22 13:04:55,952 : INFO : min_count=5 retains 15096 unique words (19% of original 75903, drops 60807)
2017-06-22 13:04:55,953 : INFO : min_count=5 leaves 1304889 word corpus (93% of original 1392037, drops 87148)
2017-06-22 13:04:55,995 : INFO : deleting the raw counts dictionary of 75903 items
2017-06-22 13:04:56,000 : INFO : sample=0.001 downsamples 45 most-common words
2017-06-22 13:04:56,001 : INFO : downsampling leaves estimated 1011849 word corpus (77.5% of prior 1304889)
2017-06-22 13:04:56,002 : INFO : estimated required memory for 15096 words and 500 dimensions: 67976000 bytes
2017-06-22 13:04:56,053 : INFO : rese

My calculation executed (normal_acc_avg) for 1 time in 173.088806 seconds seconds
My calculation best time was 173.088806 seconds seconds


0.57123727905731114

In [474]:
iter_num=3
frac_list = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
acc_avg_list = []
for frac in frac_list:
    print('Training with {}% of noise'.format(frac*100))
    acc_avg = 0
    for i in range(iter_num):
        print('\t Iter {}...'.format(i+1))
        model, df = train_with_noise(documents, targets, frac=frac)
        acc, df_result = accuracy(model, doc_test, target_test)
        acc_avg += acc
    acc_avg_list.append(acc_avg / float(iter_num))


Training with 5.0% of noise
	 Iter 1


/Users/Eddie/Documents/DSSG/WDI/nosiy_label_learning/venv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

2017-06-21 17:27:18,671 : INFO : collecting all words and their counts
2017-06-21 17:27:18,680 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-06-21 17:27:19,679 : INFO : collected 83263 word types and 20 unique tags from a corpus of 7580 examples and 1365730 words
2017-06-21 17:27:19,680 : INFO : Loading a fresh vocabulary
2017-06-21 17:27:19,748 : INFO : min_count=5 retains 15083 unique words (18% of original 83263, drops 68180)
2017-06-21 17:27:19,749 : INFO : min_count=5 leaves 1270827 word corpus (93% of original 1365730, drops 94903)
2017-06-21 17:27:19,794 : INFO : deleting the raw counts dictionary of 83263 items
2017-06

	 Iter 2


2017-06-21 17:28:41,284 : INFO : collecting all words and their counts
2017-06-21 17:28:41,286 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-06-21 17:28:42,318 : INFO : collected 79350 word types and 20 unique tags from a corpus of 7580 examples and 1333129 words
2017-06-21 17:28:42,318 : INFO : Loading a fresh vocabulary
2017-06-21 17:28:42,380 : INFO : min_count=5 retains 14903 unique words (18% of original 79350, drops 64447)
2017-06-21 17:28:42,381 : INFO : min_count=5 leaves 1242538 word corpus (93% of original 1333129, drops 90591)
2017-06-21 17:28:42,425 : INFO : deleting the raw counts dictionary of 79350 items
2017-06-21 17:28:42,429 : INFO : sample=0.001 downsamples 45 most-common words
2017-06-21 17:28:42,430 : INFO : downsampling leaves estimated 965833 word corpus (77.7% of prior 1242538)
2017-06-21 17:28:42,431 : INFO : estimated required memory for 14903 words and 500 dimensions: 67107500 bytes
2017-06-21 17:28:42,479 : INFO : reset

	 Iter 3


...

2017-06-21 17:32:03,310 : INFO : collecting all words and their counts
2017-06-21 17:32:03,312 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-06-21 17:32:04,340 : INFO : collected 83174 word types and 20 unique tags from a corpus of 7580 examples and 1427447 words
2017-06-21 17:32:04,340 : INFO : Loading a fresh vocabulary
2017-06-21 17:32:04,406 : INFO : min_count=5 retains 15622 unique words (18% of original 83174, drops 67552)
2017-06-21 17:32:04,407 : INFO : min_count=5 leaves 1331666 word corpus (93% of original 1427447, drops 95781)
2017-06-21 17:32:04,450 : INFO : deleting the raw counts dictionary of 83174 items
2017-06-21 17:32:04,456 : INFO : sample=0.001 downsamples 45 most-common words
2017-06-21 17:32:04,457 : INFO : downsampling leaves estimated 1037626 word corpus (77.9% of prior 1331666)
2017-06-21 17:32:04,457 : INFO : estimated required memory for 15622 words and 500 dimensions: 70343000 bytes
2017-06-21 17:32:04,506 : INFO : rese

	 Iter 2


2017-06-21 17:34:05,490 : INFO : collecting all words and their counts
2017-06-21 17:34:05,492 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-06-21 17:34:06,496 : INFO : collected 76520 word types and 20 unique tags from a corpus of 7580 examples and 1361834 words
2017-06-21 17:34:06,496 : INFO : Loading a fresh vocabulary
2017-06-21 17:34:06,559 : INFO : min_count=5 retains 15073 unique words (19% of original 76520, drops 61447)
2017-06-21 17:34:06,560 : INFO : min_count=5 leaves 1273770 word corpus (93% of original 1361834, drops 88064)
2017-06-21 17:34:06,606 : INFO : deleting the raw counts dictionary of 76520 items
2017-06-21 17:34:06,611 : INFO : sample=0.001 downsamples 45 most-common words
2017-06-21 17:34:06,612 : INFO : downsampling leaves estimated 986533 word corpus (77.4% of prior 1273770)
2017-06-21 17:34:06,613 : INFO : estimated required memory for 15073 words and 500 dimensions: 67872500 bytes
2017-06-21 17:34:06,664 : INFO : reset

	 Iter 3


2017-06-21 17:36:05,713 : INFO : collecting all words and their counts
2017-06-21 17:36:05,715 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-06-21 17:36:06,750 : INFO : collected 82118 word types and 20 unique tags from a corpus of 7580 examples and 1399460 words
2017-06-21 17:36:06,751 : INFO : Loading a fresh vocabulary
2017-06-21 17:36:06,819 : INFO : min_count=5 retains 15459 unique words (18% of original 82118, drops 66659)
2017-06-21 17:36:06,820 : INFO : min_count=5 leaves 1305946 word corpus (93% of original 1399460, drops 93514)
2017-06-21 17:36:06,865 : INFO : deleting the raw counts dictionary of 82118 items
2017-06-21 17:36:06,871 : INFO : sample=0.001 downsamples 47 most-common words
2017-06-21 17:36:06,872 : INFO : downsampling leaves estimated 1015949 word corpus (77.8% of prior 1305946)
2017-06-21 17:36:06,873 : INFO : estimated required memory for 15459 words and 500 dimensions: 69609500 bytes
2017-06-21 17:36:06,925 : INFO : rese

In [475]:
acc_avg_list

[0.69603642206748795, 0.65979289412604891]

In [471]:
acc

0.69791108730583828

0.57284413497589715

In [27]:
import matplotlib.pyplot as plt